### This note book will do the following

>   For training a model Tune hyperparameters with a sweep job

>   With the best model that is saved Create a managed online endpoint.

>   Deploy an MLflow model.

>   Test the endpoint.

#### Getting Started

> In compute get the repo 
  
  `git clone https://github.com/PrabalMukherjee/mslearn-azure-ml.git azure-ml-labs`

> Run setup for creating compute 

  `.\setup.sh`

> Open ML workspace and in compute stand alone  terminal do the following 

  `pip install azure-ai-ml`

  `git clone https://github.com/PrabalMukherjee/mslearn-azure-ml.git azure-ml-labs`

In [ ]:
pip show azure-ai-ml

In [ ]:
# Setting up Environment
from azure.identity import InteractiveBrowserCredential, DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data
from azure.ai.ml.sweep import Choice


#### Connect to workspace       

In [ ]:
#Connect to Azure ML Workspace

try:
    credential = DefaultAzureCredential()
    credential.get_token('https://management.azure.com/.default')
    print("DefaultAzureCredential works")
except Exception as e:
    print("DefaultAzureCredential does not work", e)
    credential = InteractiveBrowserCredential()

#Get a handke to workspace
ml_client = MLClient.from_config(credential=credential)


#### Create a Data Asset for Input

In [ ]:
input_data = Data(
    name="diabetes-data",
    type=AssetTypes.URI_FILE,
    path="data/diabetes.csv",
    version="1"
    )

ml_client.data.create_or_update(input_data)

# get the data asset
data_asset_from_registry = ml_client.data.get(
    name="diabetes-data", version=1
)
print(data_asset_from_registry)

#### Configure Command Job and Submit

In [ ]:
# configure job
# path="data/diabetes.csv"

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

#### Hyper parameter tuning creating and running sweep job

In [ ]:

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

In [ ]:
# Download best trial model output
ml_client.jobs.download(sweep_job.name, output_name="model")